---
title: "Twelve Clock List"
subtitle: "Documenting the options for a clock game"
date: 2020-06-04
categories: 
  - Python
tags: 
  - pandas
slug: "twelve-clock"
image:
  caption: ''
  focal_point: ''
  preview_only: yes
links:
  donate_button:
    icon: seedling
    icon_pack: fas
    name: Ways to Support
    url: /support/
  jupyter_button:
    icon: external-link-alt
    icon_pack: fas
    name: Jupyter Notebook
    url: https://github.com/zachbogart/my_website/blob/master/content/post/2020-06-04-digital-twelve/2020-06-04-digital-twelve.ipynb
            
---

<!-- Icon Image: Small -->
<img src="featured.png" width="100"/> 

## ...What?
Whenever I see a digital clock, I do a little math in my head. I try to take the numbers and, using only addition, subtraction, multiplication and division, get them to come out to twelve. I thought I'd try to get an exhaustive list of times that this trick applies to. Let's figure it out!

In [182]:
import re
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

## Get with the times
First we need to get a list of the times. I'm going to work with strings. Let's go through some numbers and rule out invalid ones by places.

In [10]:
valid = []
for x in range(100, 1260):
    num = ''
    num += str(x)
    
    if len(num) == 3:
        if int(num[0]) <= 9 and int(num[1]) <= 5:
            valid.append(num)
    if len(num) == 4:
        if int(num[0]) <= 1 and int(num[1]) <= 2 and int(num[2]) <= 5:
            valid.append(num)

Cool. We can check that we have 720, which is half of all times since they all repeat once. Now we can go about making possible math expressions.  

In [12]:
len(valid)

720

In [13]:
valid[:4]

['100', '101', '102', '103']

## Prep the times
To do this, I'm going to put together strings of math expressions for all permutations of the numbers, then evaluate them and see which ones work. Certainly brute force, but it'll work. First, I make a dictionary of all permutations of the numbers for each time.

In [67]:
# moded from https://www.geeksforgeeks.org/python-permutation-given-string-using-inbuilt-function/
from itertools import permutations 

def allPermutations(str): 
    
    perm_array = []

    # Get all permutations of string 'ABC' 
    permList = permutations(str) 

    # print all permutations 
    for perm in list(permList): 
        result = ''.join(perm)
        perm_array.append(result)
    
    return list(set(perm_array))
        

In [73]:
time_dict = {}

for time in valid:
    time_dict[time] = allPermutations(time)

In [113]:
time_dict['957']

['795', '957', '579', '597', '759', '975']

Now, I go and add in all possible combinations of operators (`+, -, *, /`) in between the numbers. I also add in parens since `eval` follows PEMDAS and we want to avoid that in many cases.

In [99]:
equations = {}

for time, perms in time_dict.items():
    eq = ''
    ops = '+-*/'
    if len(time) == 3:
        equations[time] = []
        for perm in perms:
            for a in ops:
                for b in ops:
                    eq = f"{perm[0]}{a}{perm[1]}{b}{perm[2]}"
                    eq = re.sub(r'([0-9])', r'\1)', eq)
                    eq = '(((' + eq
                    equations[time].append(eq)
    if len(time) == 4:
        equations[time] = []
        for perm in perms:
            for a in ops:
                for b in ops:
                    for c in ops:
                        eq = f"{perm[0]}{a}{perm[1]}{b}{perm[2]}{c}{perm[3]}"
                        eq = re.sub(r'([0-9])', r'\1)', eq)
                        eq = '((((' + eq
                        equations[time].append(eq)
                            

In [128]:
equations['100'][:10]

['(((0)+0)+1)',
 '(((0)+0)-1)',
 '(((0)+0)*1)',
 '(((0)+0)/1)',
 '(((0)-0)+1)',
 '(((0)-0)-1)',
 '(((0)-0)*1)',
 '(((0)-0)/1)',
 '(((0)*0)+1)',
 '(((0)*0)-1)']

## Look at the time

Cool. Now that we have a bunch of expressions, let's evaluate them and see what we get.

In [122]:
they_work = {}

for time, eq_list in equations.items():
    for eq in eq_list:
        try:
            result = eval(eq)
        except ZeroDivisionError:
            continue
        if result == 12 and time not in they_work.keys():
            they_work[time] = eq

In [123]:
they_work

{'116': '(((1)+1)*6)',
 '124': '(((2)+1)*4)',
 '125': '(((5)+1)*2)',
 '126': '(((6)*1)*2)',
 '127': '(((7)-1)*2)',
 '129': '(((9)+1)+2)',
 '133': '(((1)+3)*3)',
 '134': '(((1)*4)*3)',
 '135': '(((5)-1)*3)',
 '136': '(((3)-1)*6)',
 '138': '(((1)+3)+8)',
 '139': '(((3)+9)*1)',
 '142': '(((2)+1)*4)',
 '143': '(((1)*4)*3)',
 '144': '(((4)-1)*4)',
 '147': '(((7)+4)+1)',
 '148': '(((8)+4)*1)',
 '149': '(((9)+4)-1)',
 '152': '(((5)+1)*2)',
 '153': '(((5)-1)*3)',
 '156': '(((6)+1)+5)',
 '157': '(((1)*7)+5)',
 '158': '(((5)-1)+8)',
 '206': '(((2)+0)*6)',
 '214': '(((2)+1)*4)',
 '215': '(((5)+1)*2)',
 '216': '(((6)*1)*2)',
 '217': '(((7)-1)*2)',
 '219': '(((9)+1)+2)',
 '223': '(((3)*2)*2)',
 '224': '(((2)+4)*2)',
 '225': '(((5)*2)+2)',
 '227': '(((7)*2)-2)',
 '228': '(((2)+2)+8)',
 '232': '(((3)*2)*2)',
 '233': '(((3)+3)*2)',
 '236': '(((2)*3)+6)',
 '237': '(((2)+3)+7)',
 '238': '(((3)/2)*8)',
 '239': '(((9)-3)*2)',
 '241': '(((2)+1)*4)',
 '242': '(((2)+4)*2)',
 '244': '(((4)*2)+4)',
 '245': '((

In [133]:
print(f"So this trick works {len(they_work.keys()) / len(valid) * 100:.2f}% of the time")

So this trick works 45.97% of the time


## Well...there ya go
There are a bunch that are easy to spot. Was interesting to see ones where division is used. 

In [135]:
[x for x in they_work.values() if '/' in x]

['(((3)/2)*8)',
 '(((3)/2)*8)',
 '(((9)/3)*4)',
 '(((9)/3)*4)',
 '(((6)*8)/4)',
 '(((3)/2)*8)',
 '(((8)/2)+8)',
 '(((3)/2)*8)',
 '(((6)*8)/4)',
 '(((9)/3)*4)',
 '(((9)/3)+9)',
 '(((9)/3)*4)',
 '((((2)/2)+1)*6)',
 '((((2)+1)*8)/2)',
 '((((9)-1)*3)/2)']

Maybe you'll give it a try next time you spot a digital clock...

## DataFrame of the results

In [170]:
df = pd.DataFrame(valid)
df.rename(columns={0: 'raw'}, inplace=True)

In [171]:
df['raw_temp'] = df['raw'].apply(lambda x: '0' + x if len(x) == 3 else x)

In [172]:
df['time'] = df['raw_temp'].apply(lambda x: pd.to_datetime(x, format='%H%M'))

In [180]:
df.head()

,raw,raw_temp,time,twelve
0,100,0100,1900-01-01 01:00:00,False
1,101,0101,1900-01-01 01:01:00,False
2,102,0102,1900-01-01 01:02:00,False
3,103,0103,1900-01-01 01:03:00,False
4,104,0104,1900-01-01 01:04:00,False


In [176]:
valid_times = list(they_work.keys())

In [178]:
df['twelve'] = df['raw'].isin(valid_times)

In [181]:
df.head()

,raw,raw_temp,time,twelve
0,100,0100,1900-01-01 01:00:00,False
1,101,0101,1900-01-01 01:01:00,False
2,102,0102,1900-01-01 01:02:00,False
3,103,0103,1900-01-01 01:03:00,False
4,104,0104,1900-01-01 01:04:00,False


### Image Credit
[Twelve](https://thenounproject.com/search/?q=twelve&creator=4129988&i=2184535) by Zach Bogart from [the Noun Project](https://thenounproject.com/)